<a href="https://colab.research.google.com/github/omarn8911/Dphi-MLBootCamp/blob/master/covid_co.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scipy

# Imports
Empecemos por invocar nuestro cinturón de herramientas.


In [0]:
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
from plotly.subplots import make_subplots

from scipy.stats import ttest_ind
from statsmodels.stats import weightstats as stests
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



# Carga de datos

Vamos a cargar nuestro dataset. Los datos usados en este ejercicio fueron extraidos de KAGGLE: https://www.kaggle.com/dsresearchperu/novel-coronavirus-2019-dataset-latin-america?

Mostremos una pequeña muestra de los datos.


In [0]:
df = pd.read_csv('data_per_patient_CO.csv')
df.head(5)

FileNotFoundError: ignored

Ahora, miremos un poco más al detalle la "forma" (dimensiones) de nuestro dataset, para saber cuántos registros tenemos y de paso, analicemos las columnas que están disponibles.

In [0]:
print("Shape: ", df.shape)
print("Columnas: ", df.columns)

Shape:  (29383, 16)
Columnas:  Index(['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA',
       'Ciudad de ubicación', 'Departamento o Distrito', 'atención', 'Edad',
       'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS',
       'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado',
       'fecha reporte web'],
      dtype='object')


# 0 - Hipótesis

Teniendo una visión panorámica de los datos, este es un buen momento para detenernos y plantear los objetivos (hipótesis) a resolver en este ejercicio teniendo en cuenta los datos disponibles. Es importante que estas preguntas se planteen en términos de los datos disponibles:


1. El coronavirus ataca más a los mayores de 60 años. (EDAD)

2. Las grandes ciudades son las más afectadas (CIUDAD DE UBICACIÓN)

3. Los hombres son más propensos a contagiarse que las mujeres (SEXO) 



Por ahora, no nos dejemos agobiar por los datos. Tampoco por ser demasiado rigurosos con un primer acercamiento a nuestras hipótesis, **esto también es un proceso iterativo e incremental que se irá puliendo con más trabajo**.  Examinemos las columnas y decidamos cuáles nos pueden llegar a servir.

# Análisis exploratorio de los datos y visualización

Usemos la función info() de pandas para tener una perspectiva de las columnas, los valores nulos y los tipos de dato.

Luego miremos cuántos valores únicos existen en cada una de las columnas.

Nota: Las columnas de Dtype object son columnas categóricas.


In [0]:
print(df.info())
print("\nConteo de valores únicos:\n")
print(df.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29383 entries, 0 to 29382
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID de caso               29383 non-null  int64         
 1   Fecha de notificación    29383 non-null  datetime64[ns]
 2   Codigo DIVIPOLA          29383 non-null  int64         
 3   Ciudad de ubicación      29383 non-null  object        
 4   Departamento o Distrito  29383 non-null  object        
 5   atención                 29339 non-null  object        
 6   Edad                     29383 non-null  int64         
 7   Sexo                     29383 non-null  object        
 8   Tipo                     29383 non-null  object        
 9   Estado                   29333 non-null  object        
 10  País de procedencia      28236 non-null  object        
 11  FIS                      29380 non-null  object        
 12  Fecha de muerte          26959 n

Vemos que la columna de fechas no está en formato **datetime**
entonces hacemos la conversión correspondiente. 

In [0]:
df["Fecha de notificación"] = df["Fecha de notificación"].astype("datetime64")

#Plotly express para "gráficos de una sola línea"
fig = px.bar(df, x='Fecha de notificación', y='ID de caso')
fig.update_traces(marker_color='green')
fig.show()

NameError: ignored

# 1 - Análisis por edad

In [0]:
df['Edad'].describe()

count    29383.000000
mean        39.053296
std         18.714368
min          0.000000
25%         26.000000
50%         37.000000
75%         52.000000
max        103.000000
Name: Edad, dtype: float64

In [0]:
fig = px.histogram(df, x='Edad', y='ID de caso', histfunc='count')
fig.show()

In [0]:
fig = px.box(df, x='Edad')
fig.show()

In [0]:
df_menores = df[ df['Edad'] < 60 ]
df_mayores = df[df['Edad'] >= 60 ]

In [0]:
print("Menores: ", df_menores.shape[0])
print("Mayores: ", df_mayores.shape[0])

Menores:  24993
Mayores:  4390


In [0]:
df['Estado'].unique()

array(['Leve', 'Asintomático', 'Fallecido', 'Grave', 'Moderado', nan,
       'leve'], dtype=object)

In [0]:
df_menores['Estado'] = df_menores['Estado'].str.lower()
df_mayores['Estado'] = df_mayores['Estado'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
fig = px.histogram(df_menores, title='Menores', x='Estado', y = 'ID de caso', histfunc='count')
fig.show()

In [0]:
fig = px.histogram(df_mayores, title='Mayores', x='Estado', y = 'ID de caso', histfunc='count')
fig.show()

In [0]:
g_menores = df_menores.groupby(by='Estado')
g_mayores = df_mayores.groupby(by='Estado')

menores_estandar = {}
for estado, datos in g_menores:
  menores_estandar[estado] = round(len(datos) / df_menores.shape[0], 2)

mayores_estandar = {}
for estado, datos in g_mayores:
  mayores_estandar[estado] = round(len(datos) / df_mayores.shape[0], 2)

print("Para los menores: ", menores_estandar)
print("Para los mayores: ", mayores_estandar)

Para los menores:  {'asintomático': 0.13, 'fallecido': 0.01, 'grave': 0.01, 'leve': 0.81, 'moderado': 0.04}
Para los mayores:  {'asintomático': 0.06, 'fallecido': 0.15, 'grave': 0.02, 'leve': 0.64, 'moderado': 0.12}


In [0]:
# Fragmento de código horrible hecho a la 1 de la mañana.
# Seguramente existen formas más fáciles de hacerlo,
# Pero la vida es demsiado corta para andar de perfeccionistas.

# 1 Creamos un dataframe con los porcentajes por estado para menores
df_edad = pd.DataFrame(list(menores_estandar.items()))
# 2 Le pegamos horizontalmente (left join) los porcentajes de los mayores
df_edad = df_edad.join(pd.DataFrame(list(mayores_estandar.items())), rsuffix='r_')
# 3 La operación genera una columna repetida, hay que matarla.
df_edad.drop(labels=['0r_'], axis=1, inplace=True)
# 4 Renombramos las columnas, y definimos un índice.
df_edad.columns = ['Estado', 'Menores', 'Mayores']
df_edad.set_index('Estado', inplace=True)
# 5 ¡Sorpresa! En realidad vamos a usar la transpuesta. (Ver el print)
df_edad = df_edad.T
print(df_edad)

# Todo para un intento fallido de gráfica en paralelo.
# Pero, algo de información podemos extaer de acá...
fig = px.parallel_coordinates(df_edad, range_color=[0,1]) 
fig.show()

Estado   asintomático  fallecido  grave  leve  moderado
Menores          0.13       0.01   0.01  0.81      0.04
Mayores          0.06       0.15   0.02  0.64      0.12


Teniendo en cuenta la información desagregada que hemos recolectado, podemos concluir que en principio, aunque los menores de 60 años presentan más casos agregados, cuando miramos por los estados de cada paciente, los mayores de 60 años ganan porcentualmente en las categorías de:

1. Fallecidos
2. Graves
3. Moderados.

# 2 - Análisis por ciudad

In [0]:
df['Ciudad de ubicación'].describe()

count           29383
unique            410
top       Bogotá D.C.
freq             9989
Name: Ciudad de ubicación, dtype: object

In [0]:
df_por_ciudad = df.groupby(by='Ciudad de ubicación')

dict_sort = {}
for ciudad, data in df_por_ciudad:
  dict_sort[ciudad] = len(data)

print({k: v for k, v in sorted(dict_sort.items(), key=lambda item: item[1], reverse=True)})

{'Bogotá D.C.': 9989, 'Cartagena de Indias': 2975, 'Cali': 2615, 'Barranquilla': 2100, 'Leticia': 1805, 'Soledad': 1133, 'Villavicencio': 938, 'San Andrés de Tumaco': 728, 'Medellín': 551, 'Buenaventura': 496, 'Santa Marta': 339, 'Soacha': 315, 'Malambo': 282, 'Ituango': 200, 'Valledupar': 190, 'Quibdó': 181, 'Pereira': 176, 'Ibagué': 167, 'Neiva': 151, 'Ipiales': 143, 'Puebloviejo': 129, 'Ciénaga': 103, 'Pasto': 99, 'Sabanagrande': 95, 'Cúcuta': 90, 'Bello': 86, 'Mosquera': 85, 'Turbaco': 81, 'Galapa': 77, 'Armenia': 71, 'Dosquebradas': 69, 'Palmira': 68, 'Montería': 64, 'Tunja': 57, 'La Dorada': 56, 'Carepa': 55, 'Yumbo': 51, 'Jamundí': 49, 'Palmar de Varela': 48, 'Chía': 47, 'Manizales': 46, 'Puerto Nariño': 42, 'Santo Tomás': 41, 'San Luis': 40, 'Puerto Colombia': 39, 'Santa Rosa': 39, 'Popayán': 37, 'Funza': 36, 'Facatativá': 35, 'Fusagasugá': 35, 'Sabanalarga': 34, 'Baranoa': 33, 'Arjona': 32, 'Zipaquirá': 32, 'Envigado': 31, 'Tuluá': 31, 'Madrid': 30, 'Villanueva': 30, 'Candelar

In [0]:
fig = px.histogram(df, x='Ciudad de ubicación', y='ID de caso', histfunc='count').update_xaxes(categoryorder="total descending")
fig.show()

Con este gráfico podemos responder la segunda pregunta. Pero, para tener una respuesta que funcione es necesario contar con el consenso de los interesados.

¿Cuáles se consideran las ciudades "grandes" en Colombia?

¿El problema las abarca todas?

Si tuvieramos un critero más exacto para nuestro problema podríamos tener una conclusión más directa. Esto nos lleva a pensar que nuestra hipótesis inicial necesita ajustarse para ser más clara en su objetivo:

**"¿Las 5 ciudades más impactadas por el coronavirus son capitales de departamento?"**

# 3 - Análisis por género

Vamos a plantear esta pregunta como una hipótesis estadística.

H0: Los casos registrados entre la población de mujeres y la de hombres son estadísticamente equivalentes.

H1: Existen diferencias estadísticas significativas por género.

Siempre planteamos H0 como la hipótesis nula, la que no asume diferencias.



In [0]:
df['Sexo'].unique()

array(['F', 'M'], dtype=object)

In [0]:
df_hombres = df[ df['Sexo'] == 'M' ]
df_mujeres = df[ df['Sexo'] == 'F' ]

In [0]:

fig1 = px.box(df_hombres, x ='Edad', title='Hombres')
fig2 = px.box(df_mujeres, x ='Edad', title='Mujeres')

trace1 = fig1['data'][0]
trace2 = fig2['data'][0]

fig = make_subplots(rows=2, cols=1, shared_xaxes=False, subplot_titles=("Hombres", "Mujeres"))
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)

In [0]:
fig1 = px.histogram(df_hombres, x ='Edad', histfunc='count', title='Hombres')
fig2 = px.histogram(df_mujeres, x ='Edad', histfunc='count', title='Mujeres')

trace1 = fig1['data'][0]
trace2 = fig2['data'][0]

fig = make_subplots(rows=2, cols=1, shared_xaxes=False, subplot_titles=("Hombres", "Mujeres"))
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)

In [0]:
# Conteo de observaciones
print("Conteo hombres: ", df_hombres['Edad'].count())
print("Conteo mujeres: ", df_mujeres['Edad'].count())

# Estudiemos los promedios de las muestras
print("Promedio hombres: ", np.mean(df_hombres['Edad']))
print("Promedio mujeres: ", np.mean(df_mujeres['Edad']))

# Ahora, las desviaciones estandar.
print("Desv. estandar hombres: ", np.std(df_hombres['Edad']))
print("Desv. estandar mujeres: ", np.std(df_mujeres['Edad']))

# Para más de 30 muestras y evitar errores de Tipo II.
ztest ,pval1 = stests.ztest(df_hombres['Edad'], x2=df_mujeres['Edad'], value=0, 
                            alternative='two-sided')
print("p-value: ", pval1)
if pval1 < 0.05:
  print("Rechazamos la hipótesis nula H0")
else:
  print("Aceptamos la hipótesis nula H0")

Conteo hombres:  16363
Conteo mujeres:  13020
Promedio hombres:  38.75053474301778
Promedio mujeres:  39.43379416282642
Desv. estandar hombres:  18.31919661960344
Desv. estandar mujeres:  19.192000082616367
p-value:  0.0018752768431098547
Rechazamos la hipótesis nula H0


 Una vez más, tendremos que recurrir al contexto y a la experticia para decidir la respuesta correcta a esta pregunta. Sin embargo, si nos apegamos estrictamente a la prueba estadística, tendremos que rechazar la hipótesis nula y reconocer que **sí** existen diferencias estadísticas significativas en cómo el coronavirus afecta por edades a los dos sexos.